# Preparation

In [0]:
%sh

nslookup rrdemostorageacc.dfs.core.windows.net

Server: 168.63.129.16
Address:	168.63.129.16#53

Non-authoritative answer:
rrdemostorageacc.dfs.core.windows.net	canonical name = rrdemostorageacc.privatelink.dfs.core.windows.net.
Name:	rrdemostorageacc.privatelink.dfs.core.windows.net
Address: 10.1.0.4

In [0]:
tenant_id="8a7b0420-a9f1-4cb9-8567-8a5b02d42bd2"
scope="rr-demo-secret-scope"
secret="rr-demo-secret-key"
service_principal_id="15317507-66ff-48e8-89ac-fca37c38f38e"

mount_path="/mnt/test-mount"

In [0]:
#https://docs.microsoft.com/en-gb/azure/databricks/data/data-sources/azure/adls-gen2/azure-datalake-gen2-sp-access

configs = {
  "fs.azure.account.auth.type": "OAuth",
  "fs.azure.account.oauth.provider.type": "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider",
  "fs.azure.account.oauth2.client.id": service_principal_id,
 "fs.azure.account.oauth2.client.secret": dbutils.secrets.get(scope = scope, key = secret),
  "fs.azure.account.oauth2.client.endpoint": "https://login.microsoftonline.com/"+tenant_id+"/oauth2/token"
}

dbutils.fs.unmount(
     mount_point = mount_path
)
dbutils.fs.mount(
  source = "abfss://test-container@rrdemostorageacc.dfs.core.windows.net/",
  mount_point = mount_path,
  extra_configs= configs
)

/mnt/test-mount has been unmounted.
Out[2]: True

In [0]:


display(dbutils.fs.ls('/databricks-datasets'))

path,name,size
dbfs:/databricks-datasets/COVID/,COVID/,0
dbfs:/databricks-datasets/README.md,README.md,976
dbfs:/databricks-datasets/Rdatasets/,Rdatasets/,0
dbfs:/databricks-datasets/SPARK_README.md,SPARK_README.md,3359
dbfs:/databricks-datasets/adult/,adult/,0
dbfs:/databricks-datasets/airlines/,airlines/,0
dbfs:/databricks-datasets/amazon/,amazon/,0
dbfs:/databricks-datasets/asa/,asa/,0
dbfs:/databricks-datasets/atlas_higgs/,atlas_higgs/,0
dbfs:/databricks-datasets/bikeSharing/,bikeSharing/,0


In [0]:
read_format = 'json'
write_format = 'delta'
load_path = '/databricks-datasets/learning-spark-v2/iot-devices/iot_devices.json'
save_path = mount_path+'/iot-devices'
table_name = 'default.iot_devices'

# Load the data from its source.
iot_devices = spark \
  .read \
  .format(read_format) \
  .load(load_path)

# Write the data to its target.
iot_devices.write \
  .format(write_format) \
  .mode("overwrite") \
  .save(save_path)

In [0]:

display(dbutils.fs.ls(mount_path))

path,name,size
dbfs:/mnt/test-mount/loans/,loans/,0
dbfs:/mnt/test-mount/people-10m/,people-10m/,0


In [0]:
read_format = 'parquet'
write_format = 'delta'
load_path = '/databricks-datasets/learning-spark-v2/loans/loan-risks.snappy.parquet'
save_path = mount_path+'/loans'
table_name = 'default.loans'

# Load the data from its source.
loans = spark \
  .read \
  .format(read_format) \
  .load(load_path)

# Write the data to its target.
loans.write \
  .format(write_format) \
  .mode("overwrite") \
  .save(save_path)

In [0]:
%sql

CREATE DATABASE testdb;

CREATE TABLE testdb.iot_devices
  USING DELTA
  LOCATION '/mnt/test-mount/iot-devices';
  
CREATE TABLE testdb.loans
  USING DELTA
  LOCATION '/mnt/test-mount/loans';